In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.optimizers import Adam
# from keras.callbacks import LearningRateScheduler

## Logistic Regression - Comparing *ADAM* and *AMSGrad* on MNIST

Here, I tune and train logistic regression models, to recreate the empirical results of section 5 of the paper.

As specified in the paper, I fix the parameter $\beta_1$ at .99, and tune the learning rate $\alpha$ and the hyperparameter $\beta_2$ using a gridsearch, as done in the paper. 

Note that to fit a logistic regression model, I'm using a one-layer feedforward neural network (they're equivalent). This is so that I can use the nice tools (including the ADAM optimizer) already implemented in the deep learning framework Keras. 

## 0. Load MNIST Dataset

I've already created train and test splits for the MNIST dataset. They are conviniently stored as compressed numpy arrays.

In [2]:
def load_np_file(path, mode = "rb"):
    with open(path, mode) as handle:
        return(np.load(path))

In [6]:
X_train = load_np_file("../data/MNIST/X_train.npy") 
X_test = load_np_file("../data/MNIST/X_test.npy") 
y_train = load_np_file("../data/MNIST/y_train.npy") 
y_test = load_np_file("../data/MNIST/y_test.npy") 

In [7]:
# sanity check - did all the shapes get preserved? 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


## 1. A framework for exhaustive gridsearch

The hyperpameters that I'll need to tune by gridsearch are: 

- $\beta_2$
- $\alpha$.

To do so in a neat fashion, and make use of all my cores (CPU training :( ) , I'll use the `GridSearchCV` class from `sklearn`, with the `KerasClassifier` wrapper.

The interface of this wrapper requres that I define a function that can be called with a set of hyperparameter options and create a `Sequential` model that can be compiled and trained. This is what I do here. 

Note the hyperparameters that I do not tune, as they are fixed by the authors:

- $\beta_1 = .9$
- Discount rate: $\alpha_t$ = $\frac{\alpha}{\sqrt{t}}$
- Batch size = 128

In [124]:
# A function that, when passed with hyperparameter options, returns a compiled model
# Note that if `amsgrad = True`, the method in the paper is used.
def create_model(lr=0.01, beta_2 = .99, amsgrad = False):
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=784, activation='softmax'))

    
    # Compile model
    optimizer = Adam(lr=lr, beta_2 = beta_2, amsgrad = amsgrad, decay = .14)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

## 2. Gridsearch: Adam optimizer

Here I specify the ranges I'll want to look over for $\alpha$ and $\beta_2$

In [117]:
beta2_range = np.append(np.arange(.990, .999, .0025), .999)
print( beta2_range)

[0.99   0.9925 0.995  0.9975 0.999 ]


In [ ]:
alpha_range = [.001*5**i for i in range(5)]
print (alpha_range)

In [119]:
param_grid = dict(lr=alpha_range, beta_2=beta2_range)

In [125]:
adam_model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=128, verbose=1)

In [126]:
adam_model

Finally, run the gridsearch. I'll do 3-fold cross validation to choose the hyperparameters.

In [127]:
adam_grid = GridSearchCV(estimator=adam_model, param_grid=param_grid, n_jobs=-1, verbose = 3)

In [128]:
adam_grid_result = adam_grid.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] beta_2=0.99, lr=0.001 ...........................................
[CV] beta_2=0.99, lr=0.001 ...........................................
[CV] beta_2=0.99, lr=0.001 ...........................................
[CV] beta_2=0.99, lr=0.005 ...........................................


Epoch 1/100
Epoch 1/100


Epoch 1/100


40000/40000 [==============================] - 3s 71us/step - loss: 11.6424 - acc: 0.2676
Epoch 2/100
40000/40000 [==============================] - 3s 79us/step - loss: 11.3487 - acc: 0.2837
Epoch 2/100
40000/40000 [==============================] - 3s 83us/step - loss: 11.6890 - acc: 0.2622
Epoch 2/100
40000/40000 [==============================] - 2s 57us/step - loss: 10.8815 - acc: 0.3131
Epoch 3/100
40000/40000 [==============================] - 2s 58us/step - loss: 10.6989 - acc: 0.3247
Epoch 3/100
40000/40000 [==============================] - 2s 58us/step - loss: 10.1928 - acc: 0.3646
Epoch 3/100
40000/40000 [==============================] - 2s 59us/step - loss: 10.5817 - acc: 0.3308
Epoch 4/100
40000/40000 [==============================] - 2s 55us/step - loss: 10.4603 - acc: 0.3387
Epoch 4/100
40000/40000 [==============================] - 3s 63us/step - loss: 10.3855 - acc: 0.3419
Epoch 5/100
40000/40000 [==============================] - 3s 63us/step - loss: 10.2346 - acc:

/usr/local/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.101840). Check your callbacks.
  % delta_t_median)


...

40000/40000 [==============================] - 1s 36us/step - loss: 12.2409 - acc: 0.2406
Epoch 74/100
40000/40000 [==============================] - 1s 37us/step - loss: 12.3436 - acc: 0.2342
Epoch 73/100
40000/40000 [==============================] - 1s 35us/step - loss: 12.2409 - acc: 0.2406
Epoch 75/100
40000/40000 [==============================] - 2s 38us/step - loss: 13.0597 - acc: 0.1898
Epoch 65/100
40000/40000 [==============================] - 1s 36us/step - loss: 12.3436 - acc: 0.2342
Epoch 74/100
40000/40000 [==============================] - 1s 35us/step - loss: 12.2409 - acc: 0.2406
Epoch 76/100
40000/40000 [==============================] - 2s 39us/step - loss: 13.0597 - acc: 0.1898
Epoch 66/100
40000/40000 [==============================] - 1s 36us/step - loss: 12.3436 - acc: 0.2342
Epoch 75/100
40000/40000 [==============================] - 1s 35us/step - loss: 12.2409 - acc: 0.2406
Epoch 77/100
40000/40000 [==============================] - 2s 39us/step - loss: 13.05

Epoch 97/100
40000/40000 [==============================] - 1s 35us/step - loss: 12.2409 - acc: 0.2406
Epoch 99/100
40000/40000 [==============================] - 1s 35us/step - loss: 13.0597 - acc: 0.1898
Epoch 88/100
40000/40000 [==============================] - 1s 36us/step - loss: 12.3436 - acc: 0.2342
Epoch 98/100
40000/40000 [==============================] - 1s 34us/step - loss: 12.2409 - acc: 0.2406
Epoch 100/100
40000/40000 [==============================] - 1s 36us/step - loss: 13.0597 - acc: 0.1898
Epoch 89/100
40000/40000 [==============================] - 1s 35us/step - loss: 12.3436 - acc: 0.2342
Epoch 99/100
40000/40000 [==============================] - 1s 32us/step - loss: 12.3436 - acc: 0.2342
Epoch 100/100
40000/40000 [==============================] - 1s 20us/stepss: 13.0655 - acc: 0.18
[CV] ............. beta_2=0.999, lr=0.625, score=0.2382, total= 2.8min
40000/40000 [==============================] - 1s 35us/step - loss: 13.0597 - acc: 0.1898
Epoch 91/100
40000/4

[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 85.9min finished


Epoch 1/100
60000/60000 [==============================] - 1s 21us/step - loss: 8.7756 - acc: 0.4495
Epoch 2/100
60000/60000 [==============================] - 1s 19us/step - loss: 8.1835 - acc: 0.4868
Epoch 3/100
60000/60000 [==============================] - 1s 19us/step - loss: 8.0473 - acc: 0.4955
Epoch 4/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.9869 - acc: 0.4998
Epoch 5/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.9480 - acc: 0.5020
Epoch 6/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.9114 - acc: 0.5043
Epoch 7/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.8859 - acc: 0.5060
Epoch 8/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.8676 - acc: 0.5073
Epoch 9/100
60000/60000 [==============================] - 1s 18us/step - loss: 7.8531 - acc: 0.5083
Epoch 10/100
60000/60000 [==============================] - 1s 19us/step - loss: 7.8415 - a

60000/60000 [==============================] - 1s 20us/step - loss: 7.6642 - acc: 0.5204
Epoch 81/100
60000/60000 [==============================] - 2s 26us/step - loss: 7.6633 - acc: 0.5205
Epoch 82/100
60000/60000 [==============================] - 2s 26us/step - loss: 7.6623 - acc: 0.5205
Epoch 83/100
60000/60000 [==============================] - 2s 26us/step - loss: 7.6614 - acc: 0.5205
Epoch 84/100
60000/60000 [==============================] - 2s 27us/step - loss: 7.6605 - acc: 0.5207: 0s - loss: 7.661
Epoch 85/100
60000/60000 [==============================] - 2s 30us/step - loss: 7.6596 - acc: 0.5206
Epoch 86/100
60000/60000 [==============================] - 1s 25us/step - loss: 7.6588 - acc: 0.5207
Epoch 87/100
60000/60000 [==============================] - 1s 23us/step - loss: 7.6580 - acc: 0.5208
Epoch 88/100
60000/60000 [==============================] - 1s 20us/step - loss: 7.6572 - acc: 0.5209
Epoch 89/100
60000/60000 [==============================] - 1s 20us/step - lo

In [131]:
adam_grid_result.best_score_

0.6222

In [130]:
adam_grid_result.best_params_

{'beta_2': 0.9974999999999998, 'lr': 0.005}

## 3. Gridsearch - AMSGrad Optimizer

Now, repeating the same grid search, but with the AMSGrad optimizer, as described in the paper. Again, we're here to tune $\alpha$ and $\beta_2$.

In [145]:
param_grid_ams = dict(lr=alpha_range, beta_2=beta2_range, amsgrad = [True])

In [146]:
param_grid_ams

{'amsgrad': [True],
 'beta_2': array([0.99  , 0.9925, 0.995 , 0.9975, 0.999 ]),
 'lr': [0.001, 0.005, 0.025, 0.125, 0.625]}

In [147]:
ams_model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=128, verbose=1)

In [156]:
ams_grid = GridSearchCV(estimator=ams_model, param_grid=param_grid_ams, n_jobs=1, verbose = 3)

In [157]:
adms_grid_result = ams_grid.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] amsgrad=True, beta_2=0.99, lr=0.001 .............................
Epoch 1/100
40000/40000 [==============================] - 1s 26us/step - loss: 11.7513 - acc: 0.2607
Epoch 2/100
40000/40000 [==============================] - 1s 24us/step - loss: 11.2361 - acc: 0.2940
Epoch 3/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.1072 - acc: 0.3031
Epoch 4/100
40000/40000 [==============================] - 1s 23us/step - loss: 11.0305 - acc: 0.3071
Epoch 5/100
40000/40000 [==============================] - 1s 22us/step - loss: 10.9718 - acc: 0.3104
Epoch 6/100
40000/40000 [==============================] - 1s 23us/step - loss: 10.9230 - acc: 0.3134
Epoch 7/100
40000/40000 [==============================] - 1s 22us/step - loss: 10.8844 - acc: 0.3158
Epoch 8/100
40000/40000 [==============================] - 1s 29us/step - loss: 10.8518 - acc: 0.3176
Epoch 9/100
40000/40000 [===========================

40000/40000 [==============================] - 1s 29us/step - loss: 10.2817 - acc: 0.3526
Epoch 80/100
40000/40000 [==============================] - 1s 24us/step - loss: 10.2793 - acc: 0.3527
Epoch 81/100
40000/40000 [==============================] - 1s 19us/step - loss: 10.2769 - acc: 0.3529
Epoch 82/100
40000/40000 [==============================] - 1s 20us/step - loss: 10.2745 - acc: 0.3530
Epoch 83/100
40000/40000 [==============================] - 1s 19us/step - loss: 10.2722 - acc: 0.3533
Epoch 84/100
40000/40000 [==============================] - 1s 20us/step - loss: 10.2698 - acc: 0.3534
Epoch 85/100
40000/40000 [==============================] - 1s 28us/step - loss: 10.2674 - acc: 0.3535
Epoch 86/100
40000/40000 [==============================] - 1s 19us/step - loss: 10.2651 - acc: 0.3536
Epoch 87/100
40000/40000 [==============================] - 1s 19us/step - loss: 10.2628 - acc: 0.3538
Epoch 88/100
40000/40000 [==============================] - 1s 19us/step - loss: 10.26

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


Epoch 1/100
40000/40000 [==============================] - 1s 27us/step - loss: 12.9211 - acc: 0.1863
Epoch 2/100
40000/40000 [==============================] - 1s 30us/step - loss: 11.7940 - acc: 0.2546
Epoch 3/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.4195 - acc: 0.2784
Epoch 4/100
40000/40000 [==============================] - 1s 28us/step - loss: 11.2015 - acc: 0.2923
Epoch 5/100
40000/40000 [==============================] - 1s 26us/step - loss: 11.0551 - acc: 0.3016
Epoch 6/100
40000/40000 [==============================] - 1s 24us/step - loss: 10.9481 - acc: 0.3087
Epoch 7/100
40000/40000 [==============================] - 1s 20us/step - loss: 10.8552 - acc: 0.3145
Epoch 8/100
40000/40000 [==============================] - 1s 21us/step - loss: 10.7721 - acc: 0.3193
Epoch 9/100
40000/40000 [==============================] - 1s 31us/step - loss: 10.6966 - acc: 0.3235
Epoch 10/100
40000/40000 [==============================] - 1s 20us/step - loss: 1

40000/40000 [==============================] - 1s 19us/step - loss: 9.7262 - acc: 0.3852
Epoch 81/100
40000/40000 [==============================] - 1s 21us/step - loss: 9.7224 - acc: 0.3854
Epoch 82/100
40000/40000 [==============================] - 1s 22us/step - loss: 9.7185 - acc: 0.3856
Epoch 83/100
40000/40000 [==============================] - 1s 23us/step - loss: 9.7146 - acc: 0.3859
Epoch 84/100
40000/40000 [==============================] - 1s 24us/step - loss: 9.7106 - acc: 0.3860
Epoch 85/100
40000/40000 [==============================] - 1s 26us/step - loss: 9.7067 - acc: 0.3861
Epoch 86/100
40000/40000 [==============================] - 1s 30us/step - loss: 9.7029 - acc: 0.3864
Epoch 87/100
40000/40000 [==============================] - 1s 20us/step - loss: 9.6991 - acc: 0.3866
Epoch 88/100
40000/40000 [==============================] - 1s 19us/step - loss: 9.6954 - acc: 0.3868
Epoch 89/100
40000/40000 [==============================] - 1s 21us/step - loss: 9.6917 - acc: 

...

Epoch 57/100
40000/40000 [==============================] - 2s 43us/step - loss: 10.7866 - acc: 0.3299
Epoch 58/100
40000/40000 [==============================] - 2s 52us/step - loss: 10.7835 - acc: 0.3301
Epoch 59/100
40000/40000 [==============================] - 2s 49us/step - loss: 10.7792 - acc: 0.3301
Epoch 60/100
40000/40000 [==============================] - 2s 39us/step - loss: 10.7749 - acc: 0.3303
Epoch 61/100
40000/40000 [==============================] - 2s 51us/step - loss: 10.7710 - acc: 0.3307
Epoch 62/100
40000/40000 [==============================] - 2s 51us/step - loss: 10.7673 - acc: 0.3308
Epoch 63/100
40000/40000 [==============================] - 2s 41us/step - loss: 10.7638 - acc: 0.3312
Epoch 64/100
40000/40000 [==============================] - 2s 47us/step - loss: 10.7602 - acc: 0.3314
Epoch 65/100
40000/40000 [==============================] - 2s 45us/step - loss: 10.7564 - acc: 0.3315
Epoch 66/100
40000/40000 [==============================] - 2s 53us/step 

40000/40000 [==============================] - 1s 26us/step - loss: 11.2259 - acc: 0.3035
Epoch 34/100
40000/40000 [==============================] - 1s 23us/step - loss: 11.2259 - acc: 0.3035
Epoch 35/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.2259 - acc: 0.3035
Epoch 36/100
40000/40000 [==============================] - 1s 24us/step - loss: 11.2259 - acc: 0.3035
Epoch 37/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.2259 - acc: 0.3035
Epoch 38/100
40000/40000 [==============================] - 1s 24us/step - loss: 11.2259 - acc: 0.3035
Epoch 39/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.2259 - acc: 0.3035
Epoch 40/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.2259 - acc: 0.3035
Epoch 41/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.2259 - acc: 0.3035
Epoch 42/100
40000/40000 [==============================] - 1s 22us/step - loss: 11.22

40000/40000 [==============================] - 1s 23us/step - loss: 11.1519 - acc: 0.3081
Epoch 11/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1490 - acc: 0.3082
Epoch 12/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1482 - acc: 0.3083
Epoch 13/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1469 - acc: 0.3084
Epoch 14/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1466 - acc: 0.3084
Epoch 15/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1453 - acc: 0.3085
Epoch 16/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1438 - acc: 0.3086
Epoch 17/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1432 - acc: 0.3086
Epoch 18/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.1432 - acc: 0.3086
Epoch 19/100
40000/40000 [==============================] - 1s 21us/step - loss: 11.14

40000/40000 [==============================] - 2s 55us/step - loss: 11.1432 - acc: 0.3086
Epoch 90/100
40000/40000 [==============================] - 2s 48us/step - loss: 11.1432 - acc: 0.3086
Epoch 91/100
40000/40000 [==============================] - 1s 35us/step - loss: 11.1432 - acc: 0.3086
Epoch 92/100
40000/40000 [==============================] - 3s 67us/step - loss: 11.1432 - acc: 0.3086
Epoch 93/100
40000/40000 [==============================] - 2s 47us/step - loss: 11.1432 - acc: 0.3086
Epoch 94/100
40000/40000 [==============================] - 2s 41us/step - loss: 11.1432 - acc: 0.3086
Epoch 95/100
40000/40000 [==============================] - 3s 66us/step - loss: 11.1432 - acc: 0.3086
Epoch 96/100
40000/40000 [==============================] - 2s 56us/step - loss: 11.1432 - acc: 0.3086
Epoch 97/100
40000/40000 [==============================] - 1s 36us/step - loss: 11.1432 - acc: 0.3086
Epoch 98/100
40000/40000 [==============================] - 2s 56us/step - loss: 11.14

KeyboardInterrupt: 